# FDIC Data API Client

By 503 Team 4

Source:
https://banks.data.fdic.gov/docs/#/

## Code

Import libraries

In [8]:
import requests
import pandas as pd
import json
import urllib.parse


Code

In [39]:


# Endpoint
endpoint = 'https://banks.data.fdic.gov/api/'

# Define a function to get financial reporting data for all banks
def get_all_financial_reporting_data(endpoint, start_date, end_date, fields, result_limit=5000):
    offset = 0
    financials = []
    
    #while True:
    fields_encoded = urllib.parse.quote(fields, safe='')
    #url = f'{endpoint}financials?reporting_period={reporting_period}&fields={fields_encoded}&limit={result_limit}&offset={offset}'
    url = f'{endpoint}financials?filters=RISDATE:[{start_date}%20 TO %20{end_date}]&fields={fields_encoded}&limit={result_limit}&sort_by=REPDTE&sort_order=ASC'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Normalize the JSON response into a flattened DataFrame
        #df = pd.json_normalize(data, record_path=['data'])

        # Print the DataFrame
        #print(df)

        # Use the 'data' key and extract the nested 'data' values
        financial_data = [item['data'] for item in data['data']]
        financials.extend(financial_data)
        

        # Break the loop if the length of the financial data is less than the result_limit
        #if len(financial_data) < result_limit:
        #    break

        offset += result_limit
    else:
        print(f'Request failed with status code {response.status_code}')
        return None

    return pd.DataFrame(financials)

# Example usage
reporting_period = '2020-12-31'
start_date = '2022-12-01'
end_date = '2022-12-31'
fields = 'REPDTE,CERT,NAME,CITY,STNAME,ASSET,DEP,INSTTYPE,BKCLASS,STCNTY,CBSA,CITYHCR' # no spaces between!
result_limit = 10000

all_financial_reporting_data = get_all_financial_reporting_data(endpoint, start_date, end_date, fields, result_limit)
if all_financial_reporting_data is not None:
    all_financial_reporting_data = all_financial_reporting_data.sort_values('CERT')

    display(all_financial_reporting_data.head())


,CITYHCR,STCNTY,CBSA,CITY,BKCLASS,INSTTYPE,REPDTE,ASSET,STNAME,CERT,DEP,NAME,ID
849,BOSTON,Suffolk,"BOSTON-CAMBRIDGE-NEWTON, MA-NH",BOSTON,SM,CFR,20221231,298020000,MASSACHUSETTS,14,241844000,STATE STREET BANK&TRUST CO,14_20221231
3275,AUBURN,Lee,"AUBURN-OPELIKA, AL",AUBURN,SM,CFR,20221231,1023366,ALABAMA,35,952037,AUBURNBANK,35_20221231
3352,DEMOPOLIS,Marengo,None,DEMOPOLIS,NM,CFR,20221231,444822,ALABAMA,39,407949,ROBERTSON BANKING CO,39_20221231
3409,PHENIX CITY,Russell,"COLUMBUS, GA-AL",PHENIX CITY,NM,CFR,20221231,265272,ALABAMA,41,266874,PHENIX-GIRARD BANK,41_20221231
3524,,Conecuh,None,EVERGREEN,NM,CFR,20221231,76239,ALABAMA,49,70649,BANK OF EVERGREEN,49_20221231


In [40]:
display(all_financial_reporting_data.shape)
# convert the value_counts Series to a DataFrame
column_to_check = 'INSTTYPE'
display(pd.DataFrame({'col1': all_financial_reporting_data[column_to_check].value_counts().index, 'count': all_financial_reporting_data[column_to_check].value_counts().values}))



(4773, 13)

,col1,count
0,CFR,4185
1,TFR,579
2,N/A,9


In [38]:
all_financial_reporting_data['CERT'].value_counts().sort_values(ascending=False)

10004    1
10012    1
10044    1
10055    1
10057    1
        ..
9997     1
9998     1
999      1
9963     1
99       1
Name: CERT, Length: 4773, dtype: int64